In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HEP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('heparin', 20),
 ('high energy phosphate', 14),
 ('hepatocytes', 5),
 ('human epigenome project', 4),
 ('heartbeat evoked potential', 4),
 ('herba epimedii', 4),
 ('hepatoerythropoietic porphyria', 4),
 ('hepcidin', 4),
 ('hydroxyethylphosphonate', 4),
 ('hexane etoac phase', 3),
 ('hericium erinaceus polysaccharide', 3),
 ('heparosan', 3),
 ('heptylphysostigmine', 3),
 ('hydroxyethyl phosphonate', 3)]

In [ ]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Initializing INDRA bio ontology for the first time, this may take a few minutes...
INFO: [2020-10-17 01:53:47] indra.ontology.bio.ontology - Adding nodes...
INFO: [2020-10-17 01:53:56] indra.ontology.bio.ontology - Adding xrefs...
INFO: [2020-10-17 01:54:08] indra.ontology.bio.ontology - Adding hierarchy...
INFO: [2020-10-17 01:54:25] indra.ontology.bio.ontology - Building name lookup...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Finished initializing bio ontology...
INFO: [2020-10-17 01:54:31] indra.ontology.bio.ontology - Caching INDRA bio ontology at /home/ubuntu/.indra/bio_ontology/1.3/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'real time cell analyser': 'ungrounded',
  'real time cell analysis': 'ungrounded',
  'real time cell analyzer': 'ungrounded'},
 {},
 []]

In [30]:
grounding_map, names, pos_labels = grounding_map, names, pos_labels = ({'1 13 c 2 hydroxyethylpropionate': 'PUBCHEM:74689',
  '2 hydroxyethylphosphonate': 'PUBCHEM:74689',
  'heidelberg edge perimeter': 'ungrounded',
  'heparin': 'CHEBI:CHEBI:28304',
  'heparosan': 'ungrounded',
  'hepatocytes': 'MESH:D022781',
  'hepatoerythropoietic porphyria': 'MESH:D017121',
  'hepcidin': 'HGNC:15598',
  'hexane etoac phase': 'ungrounded',
  'high energy phosphate': 'ungrounded',
  'high esterified pectin': 'ungrounded',
  'high excitation pressure': 'ungrounded',
  'human epigenome project': 'ungrounded',
  'hydroxyethyl phosphonate': 'PUBCHEM:74689'},
 {'PUBCHEM:74689': '2-Hydroxyethyl phosphate',
  'CHEBI:CHEBI:28304': 'heparin',
  'MESH:D022781': 'Hepatocytes',
  'MESH:D017121': 'Porphyria, Hepatoerythropoietic',
  'HGNC:15598': 'HAMP'},
 ['CHEBI:CHEBI:28304',
  'HGNC:15598',
  'MESH:D017121',
  'MESH:D022781',
  'PUBCHEM:74689'])

In [31]:
excluded_longforms = []

In [32]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [33]:
additional_entities = {}

In [34]:
unambiguous_agent_texts = {}

In [35]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[('HGNC:33879', 'HGNC:33879', 9)]

In [20]:
intersection2

[('ungrounded', 'HGNC:33879', 0),
 ('CHEBI:CHEBI:28304', 'HGNC:33879', 0),
 ('MESH:D017121', 'HGNC:33879', 0),
 ('PUBCHEM:74689', 'HGNC:33879', 0),
 ('MESH:D022781', 'HGNC:33879', 0),
 ('HGNC:15598', 'HGNC:33879', 0)]

In [21]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['HEP', 'DNLZ', 'zinc finger',
                                                             'C9orf151', 'ZIM17'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [22]:
names.update(additional_entities)
pos_labels.extend(additional_entities)

In [36]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-26 02:28:26] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-26 02:28:27] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.8002267573696145 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [37]:
classifier.stats

{'label_distribution': {'ungrounded': 19,
  'CHEBI:CHEBI:28304': 18,
  'MESH:D017121': 3,
  'PUBCHEM:74689': 3,
  'MESH:D022781': 4,
  'HGNC:15598': 4},
 'f1': {'mean': 0.800227, 'std': 0.124783},
 'precision': {'mean': 0.790238, 'std': 0.11839},
 'recall': {'mean': 0.819048, 'std': 0.138505},
 'CHEBI:CHEBI:28304': {'f1': {'mean': 0.813492, 'std': 0.144915},
  'pr': {'mean': 0.85, 'std': 0.2},
  'rc': {'mean': 0.793333, 'std': 0.111853}},
 'MESH:D022781': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'HGNC:15598': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'PUBCHEM:74689': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'MESH:D017121': {'f1': {'mean': 0.6, 'std': 0.489898},
  'pr': {'mean': 0.6, 'std': 0.489898},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'unground

In [38]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [40]:
disamb.dump(model_name, results_path)

In [39]:
print(disamb.info())

Disambiguation model for HEP

Produces the disambiguations:
	2-Hydroxyethyl phosphate*	PUBCHEM:74689
	HAMP*	HGNC:15598
	Hepatocytes*	MESH:D022781
	Porphyria, Hepatoerythropoietic*	MESH:D017121
	heparin*	CHEBI:CHEBI:28304

Class level metrics:
--------------------
Grounding                      	Count	F1     
                     Ungrounded	19	0.83143
                        heparin*	18	0.81349
                    Hepatocytes*	 4	    0.6
                           HAMP*	 4	    0.4
Porphyria, Hepatoerythropoietic*	 3	    0.6
       2-Hydroxyethyl phosphate*	 3	    0.6

Weighted Metrics:
-----------------
	F1 score:	0.80023
	Precision:	0.79024
	Recall:		0.81905

* Positive labels
See Docstring for explanation



In [41]:
model_to_s3(disamb)